In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score
import utils.matched_roi as mr
import utils.merged_df_annotation as mda
import utils.population_activity as pa
from importlib import reload

base_dir = Path(r'E:\TPM\JK\h5')

expert_mice_df = pd.read_csv(base_dir / 'expert_mice.csv', index_col=0)
use_mice_df = expert_mice_df.loc[expert_mice_df['depth_matched'].astype(bool) & 
                                 ~expert_mice_df['processing_error'].astype(bool) &
                                 ((expert_mice_df.session_type == 'training') |
                                  (expert_mice_df.session_type.str.contains('test')))]

mice = [25,27,30,36,39,52]

In [2]:
# All the sessions
# Takes about 51 min

touch_window='after_answer'
spk_norm='std'
varexp_threshold = 0.05
post_touch_frames=1
run_mouse = []
run_volume = []
run_session = []
explained_variances_fit = []
explained_variances_all = []
num_trials = []
num_cells_all = []
num_cells_fit = []
angles = np.arange(45, 150, 15)
for mi in range(len(mice)):
    mouse = mice[mi]
    for volume in range(1,3):
        top_plane = 1 if volume == 1 else 5
        sessions_str = use_mice_df.query('mouse==@mouse and plane==@top_plane').session.values
        for session_str in sessions_str:
            session = int(session_str)
            
            if len(use_mice_df.query('mouse == @mouse and plane == @top_plane and session == @session_str')) > 0:
                per_touch_response_xr_fit, per_touch_response_df, per_touch_response_xr = \
                    pa.get_touch_response_xr_varexp_threshold(base_dir, mouse, top_plane, session,
                                                        touch_window=touch_window, 
                                                        spk_norm=spk_norm,
                                                        varexp_threshold=varexp_threshold,
                                                        post_touch_frames=post_touch_frames)
                num_cells_all.append(per_touch_response_xr.shape[1])
                num_cells_fit.append(per_touch_response_xr_fit.shape[1])
                pca = PCA()
                pca.fit(per_touch_response_xr_fit.values)
                explained_variances_fit.append(pca.explained_variance_ratio_)
                pca = PCA()
                pca.fit(per_touch_response_xr.values)
                explained_variances_all.append(pca.explained_variance_ratio_)

                temp_num_trials = []
                for angle in angles:
                    temp_num_trials.append(len(per_touch_response_df.query('pole_angle == @angle')))
                num_trials.append(temp_num_trials)
                run_mouse.append(mouse)
                run_volume.append(volume)
                run_session.append(session)
expvar_results = pd.DataFrame({'mouse': run_mouse, 'volume': run_volume, 'session': run_session,
                                 'explained_variances_fit': explained_variances_fit,
                                 'explained_variances_all': explained_variances_all,
                                 'num_trials': num_trials,
                                 'num_cells_fit': num_cells_fit,
                                 'num_cells_all': num_cells_all,
                                 })

results_dir = Path(r'E:\TPM\JK\h5\results\neural_stretching\lda_performances')
expvar_results.to_pickle(results_dir / 'explained_variances_expert_all_sessions_after_answer.pkl')

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [15]:
per_touch_response_xr_fit, per_touch_response_df, per_touch_response_xr = \
                    pa.get_touch_response_xr_varexp_threshold(base_dir, mouse, top_plane, session,
                                                        touch_window=touch_window, 
                                                        spk_norm=spk_norm,
                                                        varexp_threshold=varexp_threshold,
                                                        post_touch_frames=post_touch_frames)

In [20]:
len(np.where(per_touch_response_df.num_touch==0)[0])

10

In [13]:
11*283

3113

In [21]:
volume

1

In [33]:
plane = 4
touch_window = 'after_answer'
plane_dir = base_dir / f'{mouse:03}/plane_{plane}'
behavior_frametime = pd.read_pickle(plane_dir / f'JK{mouse:03}_S{session:02}_plane{plane}_frame_whisker_behavior.pkl')
roi_dir = plane_dir / f'{session:03}/plane0/roi'
ophys_frametime = pd.read_pickle(roi_dir / 'refined_frame_time.pkl')

# get merged df
merged_df = mda.get_merged_df(ophys_frametime, behavior_frametime)
# assign pole_moving_up and pole_moving_down to the frames
merged_df = mda.assign_pole_moving_frames(merged_df)

# get spks
norm_spks = mda.get_normalized_spikes(roi_dir, ophys_frametime, merged_df, spk_norm=spk_norm)

merged_df['spks_frame_ind'] = np.arange(norm_spks.shape[1])

# get touch response frames
merged_df = mda.assign_touch_response_frames(merged_df, post_touch_frames=post_touch_frames)

# Get touch response of spks per trial
if touch_window == 'before_answer':
    touch_trial_nums = merged_df[merged_df.before_answer_touch_frame].trialNum.unique()
elif touch_window == 'after_answer':
    touch_trial_nums = merged_df[merged_df.after_answer_touch_frame].trialNum.unique()
else:
    touch_trial_nums = merged_df[merged_df.touch_response_frame].trialNum.unique()
merged_df = merged_df[merged_df.trialNum.isin(touch_trial_nums)]
merged_df['num_touch'] = np.nan

In [34]:
np.where(merged_df.groupby('trialNum').apply(lambda x: sum(x.after_answer_touch_frame.values)).values==0)

(array([], dtype=int64),)

In [25]:
len(merged_df.trialNum.unique())

56

In [24]:
len(touch_trial_nums)

56

In [11]:
len(np.unique(np.where(np.isinf(per_touch_response_xr_fit.values))[0]))

11

In [14]:
len(np.where(np.isinf(per_touch_response_xr_fit.values))[0])

2131